# ClinicalProteomic 

In [ ]:
#En primer lugar importaremos aquellas librerías que necesitaremos para realizar la práctica:  
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

from sklearn.model_selection import train_test_split

In [ ]:
#Leeremos el fichero csv que nos proporciona el consorcio para así poder realizar el tratamiento de los datos: 
can_p1 = pd.read_csv('../Data/breast_cancer_data.csv')
can_p1

Realizamos una primera vista de los datos que aparecen en el fichero de Breast Cancer Data. Como podemos observar, hay valor que aparecen como nulos, según el consorcio sanitario estos valor se clasifican como proteínas que no se habrán encontrado durante el analisi. Empezaremos el proceso de ETL, borrando todos aquellos valores nulos. 

In [ ]:
#Identificaremos aquellos valores en NaN: 
can_p1.isna()
#Los reemplazamos:
can_p1 = can_p1.fillna(0)
can_p1.describe()

Dada que las tres primeras columnas de nuestro data frame son muy extensas tienen cada una de ellas más de 10000 filas, la eliminaremos. Si quisieramos coonvertirlas en variables explicativas dado que tienen variable texto tendríamos muchísimas columnas para cada uno de los valores que incorporan cada uno de estos campos.  

In [ ]:
#Borraremos los tres primeros campos de nuestro df nombrado "can_p1". 
df_cancer = can_p1.drop(['RefSeq_accession_number', 'gene_symbol','gene_name'],axis = 1)
df_cancer

In [ ]:
#Verificaremos que todos los campos realmente tienen la misma longitud: 
df_cancer.describe()

# Proceso de Clustering

El objetivo de nuestra práctica será realizar un proceso de clusterización. En él, nos interesará poder clasificar aquellas muestras de tumores que sean parecidas para poder facilitar el estudio de los distintos cánceres de mama. 
Véase a continuación, el plroceso por el cuál queremos definit los distintos clústers que se pueden identificar en nuestro data set. 

Dado que no sabemos cuantos serán el número de clusters ideal para nuestro modelo utilizaremos el algoritmo 'elbow'. Así pues sabremos más o menos el número de agrupaciones que debemos utilizar. 

In [ ]:
#We need to find with kmeans elbow method the optimal value for 'k' (#clusters):

#1. Primero importaremos las librerías necesarias para trabajar con este método: 
from sklearn.cluster import KMeans
from sklearn import metrics
from scipy.spatial.distance import cdist

#2. Para determinar el número eficiente de k veremos: 
distortions = []
K = range(1,10)
for k in K:
    kmeanModel = KMeans(n_clusters=k).fit(df_cancer)
    kmeanModel.fit(df_cancer)
    distortions.append(sum(np.min(cdist(df_cancer, kmeanModel.cluster_centers_, 'euclidean'), axis=1)) / df_cancer.shape[0])

In [ ]:
#Graficaremos el método elbow: 
plt.plot(K, distortions, 'bx-')
plt.xlabel('k')
plt.ylabel('Distortion')
plt.title('The Elbow Method showing the optimal k')
plt.show()


(*1) Ahora, necesitaremos utilizar el modelo Kmeans. Arriba podemos observar el gráfico 'The Elbow Method Showing the optimal k', así pues, podemos ver que el valor óptimo de k es de más o menos 6. 


Véase a continuación que necesitaremos modificar nuestro modelo de machine learning. Para eso utilizaremos la función Principal Component Analysis (PCA). Con esto queremos hacer que nuestro algoritmo de predicción sea más ágil, dado que los inputs procedentes de nuestro dataset son muchos. 

In [ ]:
#Primero tenemos que normalizar los datos de nuestra dataset: 

from sklearn import preprocessing 
from sklearn.cluster import KMeans

In [ ]:
#Normalizamos: 
min_max_scaler = preprocessing.MinMaxScaler()
df_cancernorm = min_max_scaler.fit_transform(df_cancer)
df_cancernorm  = pd.DataFrame(df_cancer)
df_cancernorm

In [ ]:
#Utilizaremos PCA en python para poder juntar la columnas y definir una variable X y otra Y para poder realizar el modelo de

from sklearn.decomposition import PCA

pca = PCA(n_components = 2)
principalComponents = pca.fit_transform(df_cancernorm)
principalDf = pd.DataFrame(data = principalComponents
                           , columns = ['principal component 1', 'principal component 2'])
principalDf

In [ ]:
#Representamos la distribución de nuestro data set sobre el cancer de mama, dada la agrupación realizada a partir de la función PCA: 

x = principalDf['principal component 1'].values
y = principalDf['principal component 2'].values
plt.xlabel('Component 1')
plt.ylabel('Component 2')
plt.title('Distribución de nuestro dataset')
plt.plot(x,y,'o',markersize=1)

In [ ]:
#Aplicación del modelo K-means k = 6. Véase la nota (*1). 
kmeans_model = KMeans(n_clusters=6).fit(principalDf)
cluster_model = kmeans_model.cluster_centers_
print(cluster_model)

#Véase que la función aplicada nos mostrá las coordenadas de cada uno de los centros de los clústers que se forman. 

In [ ]:
#Etiquetaaremos los datos. Véase que asignaremos cada uno de nuestros registros a uno de los clústers que hemos encontrado: 

labels = kmeans_model.predict(principalDf)

Una vez hemos asignado nuestros registros a cada uno de los clústers que hemos definido, realizaremos una representación gráfica de estos para poder realizar una análsis visual. 

In [ ]:
#Representación gráfica de los clústers de kmeans: 

colores=['red','green','blue','yellow','fuchsia','black']
asignar=[]
for row in labels:
     asignar.append(colores[row])
plt.scatter(x, y, c=asignar, s=1)
plt.scatter(cluster_model[:, 0], cluster_model[:, 1], marker='*', c='black', s=20)
plt.xlabel('Component 1')
plt.ylabel('Component 2')
plt.title('Representación clústers k-means')
plt.show()
